In [2]:
!pip install -q tensorflow

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [25]:
# Load and preprocess data
train_data = pd.read_csv('neumf_train_data.csv')
train_data['verified_purchase'] = np.where(train_data['verified_purchase'] == 'Y', 1, 0)

test_data = pd.read_csv('neumf_test_data.csv')
test_data['verified_purchase'] = np.where(test_data['verified_purchase'] == 'Y', 1, 0)

num_users = len(train_data.customer_id.unique()) + len(test_data.customer_id.unique())
num_items = len(train_data.product_id.unique()) + len(test_data.product_id.unique())

In [19]:
train_data.head()

,customer_id,product_id,star_rating,verified_purchase
0,6552,4052,1.0,0
1,7179,579,0.0,0
2,2160,3996,0.0,0
3,611,4351,0.0,0
4,3016,3836,0.0,0


In [20]:
# Define NeuMF model architecture
embedding_dim = 32
num_layers = 3
hidden_dim = 64

#input layer
user_input = Input(shape=(1,))
user_embedding_mlp = Flatten()(Embedding(num_users, embedding_dim)(user_input))
user_embedding_mf = Flatten()(Embedding(num_users, embedding_dim)(user_input))

item_input = Input(shape=(1,))
item_embedding_mlp = Flatten()(Embedding(num_items, embedding_dim)(item_input))
item_embedding_mf = Flatten()(Embedding(num_items, embedding_dim)(item_input))

mlp_layers = Concatenate()([user_embedding_mlp, item_embedding_mlp])
for i in range(num_layers):
    mlp_layers = Dense(hidden_dim, activation='relu')(mlp_layers)
    mlp_layers = Dropout(0.5)(mlp_layers)

mf_layers = tf.keras.layers.Multiply()([user_embedding_mf, item_embedding_mf])
# mf_layers = Concatenate()([user_embedding_mf, item_embedding_mf])
mf_layers = Dense(hidden_dim, activation='relu')(mf_layers)

rating_output = Concatenate()([mlp_layers, mf_layers])
rating_output = Dense(1, activation='sigmoid')(rating_output)

verified_purchase_output = Concatenate()([mlp_layers, mf_layers])
verified_purchase_output = Dense(1, activation='sigmoid')(verified_purchase_output)

In [21]:
# Define NeuMF model
neumf_model = Model(inputs=[user_input, item_input], outputs=[rating_output, verified_purchase_output])
neumf_model.load_weights('neumf_model_weights.h5')

neumf_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001)) 

In [13]:
# Train NeuMF model
neumf_model.fit(x=[train_data['customer_id'], train_data['product_id']],
                y=[train_data['star_rating'], train_data['verified_purchase']],
                batch_size=64,
                epochs=10,
                validation_data=([test_data['customer_id'], test_data['product_id']],
                                 [test_data['star_rating'], test_data['verified_purchase']]))

neumf_model.save_weights('neumf_model_weights.h5')

Epoch 1/10
510/510 [==============================] - 9s 17ms/step - loss: -25377.0059 - dense_4_loss: -25377.0059 - dense_5_loss: 7.9239e-06 - val_loss: -226607.4844 - val_dense_4_loss: -226607.4844 - val_dense_5_loss: 4.7375e-12
Epoch 2/10
510/510 [==============================] - 10s 19ms/step - loss: -165176.4375 - dense_4_loss: -165176.4375 - dense_5_loss: 1.5893e-09 - val_loss: -1486048.1250 - val_dense_4_loss: -1486048.1250 - val_dense_5_loss: 1.5297e-26
Epoch 3/10
510/510 [==============================] - 9s 18ms/step - loss: -939327.8750 - dense_4_loss: -939327.8750 - dense_5_loss: 3.6577e-17 - val_loss: -6700751.0000 - val_dense_4_loss: -6700751.0000 - val_dense_5_loss: 0.0000e+00
Epoch 4/10
510/510 [==============================] - 8s 16ms/step - loss: -3184394.2500 - dense_4_loss: -3184394.2500 - dense_5_loss: 9.5863e-30 - val_loss: -19071974.0000 - val_dense_4_loss: -19071974.0000 - val_dense_5_loss: 0.0000e+00
Epoch 5/10
510/510 [==============================] - 10s 1

In [27]:
def recommend_items(user_id, num_items=10):
    item_ids = np.arange(num_items)
    user_ids = np.repeat(user_id, num_items)
    rating_predictions, verified_purchase_predictions = neumf_model.predict([user_ids, item_ids])
    item_ratings = list(zip(item_ids, rating_predictions.flatten(), verified_purchase_predictions.flatten()))
    # item_ratings = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings
    #                 if verified_purchase >= 0.5 and rating > 3.0]
    item_ratings = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings]
    item_ratings.sort(key=lambda x: x[1], reverse=True)
    recommended_items = [(item_id, rating, verified_purchase) for item_id, rating, verified_purchase in item_ratings[:num_items]]
    return recommended_items

In [24]:
neumf_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 1, 32)        259840      ['input_3[0][0]']                
                                                                                                  
 embedding_6 (Embedding)        (None, 1, 32)        237664      ['input_4[0][0]']                
                                                                                            

In [28]:
res = recommend_items(user_id=train_data.customer_id[0])
print(res)

1/1 [==============================] - 0s 32ms/step
[(0, 1.0, 0.0), (1, 1.0, 0.0), (2, 1.0, 0.0), (3, 1.0, 0.0), (4, 1.0, 0.0), (5, 1.0, 0.0), (6, 1.0, 0.0), (8, 1.0, 0.0), (9, 1.0, 0.0), (7, 0.0, 0.0)]
